## Project: Decision Tracker
## Making informed decisions based on personal data
**Source:** Own files

### 1. Setting up environment

#### 1.1 Importing packages & setting-up parameters

In [1]:
# Import parent directory (main project directory)
# for packages importing
import sys
import os

# Getting the parent directory name in which your script is running
parent = os.path.dirname(os.path.realpath('/'))

# adding the parent directory to
# the sys.path.
sys.path.append(parent)

# now we can import the module in the parent
# directory.


In [2]:
import requests # to download data from api
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import pathlib # for path functions
import numpy as np

import sqlite3
import seaborn as sns
import ipywidgets as widgets
from IPython.display import clear_output, display

#turning on plot display in JN
%matplotlib inline 
# Setting pandas display options
pd.options.display.max_columns = 300
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 250


In [3]:
# PARAMETERS
plt.style.use('fivethirtyeight')


In [4]:
DataFolderPath = pathlib.Path(os.path.dirname(os.path.realpath(''))).parent
print(DataFolderPath)


C:\Users\tgusc\Documents\GitHub


In [5]:
#CLASSES


In [6]:
def import_bills(filename, sheet_name):
    # Necessary requirements - DataFolderPath
    file = os.path.join(DataFolderPath, 'Private', filename)
    data_in = pd.read_excel(file, sheet_name = sheet_name , decimal=',')
    data_out = data_in
    # Cleaning data
    data_out = data_out[data_out["Rok"] != 1900].copy() # usuwanie wierszy bez dat
    data_out = data_out[data_out["Kategoria"].notna()].copy()

    # Changing types
    data_out["LP"] = data_out["LP"].astype(np.int32)
    data_out["Rok"] = data_out["Rok"].astype(np.int32)
    data_out["Miesiac"] = data_out["Miesiac"].astype(np.int32)
    data_out["Data"] = pd.to_datetime(data_out["Data"])
    data_out["Cena"] = data_out["Cena"].replace(',','.').astype(float)
    if "Nr koszyka" in data_out.columns:
        data_out["Nr koszyka"] = data_out["Nr koszyka"].astype(str)

    def clean_category(category):
        return category.lower().strip()

    # Oczyszczanie nazw kategorii
    data_out['Kategoria'] = data_out['Kategoria'].apply(clean_category)

    print(data_out.tail()) # sample check
    return data_out



In [7]:
def sql_sum(data_in):
    # 2. Stworzenie lokalnej bazy danych SQLite
    conn = sqlite3.connect('transactions_db.sqlite')
    data_in.to_sql('transactions', conn, if_exists='replace', index=False)
    conn.commit()

    # Przygotowanie agregacji danych
    query_sum = "SELECT Rok, Miesiac, Rodzaj, Kategoria, SUM(Cena) AS Suma FROM transactions GROUP BY Rok, Miesiac, Kategoria, Rodzaj"
    data_out = pd.read_sql(query_sum, conn)
    print(data_out.tail())
    return data_out


In [8]:
class WidgetBarPlot():
    def __init__(self, data_in):
        self.data_in = data_in
        self._data_filters()
    
    def _data_filters(self):
        self._years = sorted(self.data_in['Rok'].unique())
        self._months = sorted(self.data_in['Miesiac'].unique())
        self._rodzaje = sorted(self.data_in['Rodzaj'].unique())
        self._kategorie = sorted(self.data_in['Kategoria'].unique())  # Nowy filtr dla kategorii
        
        self.default_year = max(self._years)
        self.default_rodzaj = "Debit"
        self.default_month = max(self.data_in[self.data_in['Rok'] == self.default_year]["Miesiac"])
        self.default_kategoria = self._kategorie  # Domyślnie wszystkie kategorie są zaznaczone

    def _barplot(self, years, months, rodzaje, kategorie):
        # Filtrujemy dane według wybranych lat, miesięcy, rodzajów transakcji i kategorii
        data = self.data_in[
            self.data_in['Rok'].isin(years) & 
            self.data_in['Miesiac'].isin(months) & 
            self.data_in['Rodzaj'].isin(rodzaje) &
            self.data_in['Kategoria'].isin(kategorie)  # Dodatkowy filtr dla kategorii
        ].sort_values('Suma')
        
        plt.figure(figsize=(10, 8))
        ax = sns.barplot(data=data, x='Miesiac', y='Suma', hue='Kategoria', order=data['Miesiac'].unique())
        
        # Dodajemy etykiety do słupków
        for p in ax.patches:
            ax.annotate(f'{p.get_height():.0f}', 
                        (p.get_x() + p.get_width() / 2., p.get_height() - p.get_height()/10), 
                        ha='center', va='center', 
                        xytext=(0, -10), 
                        textcoords='offset points')
        
        plt.title('Miesięczne wpływy/koszty w podziale na kategorie')
        
        # Przenoszenie legendy pod wykres
        ax.legend(title='Kategoria', bbox_to_anchor=(0.5, -0.2), loc='upper center', ncol=3)
        
        plt.tight_layout()
        plt.show()

    def widget_plot(self):
        # Uaktualnienie widgetu z wielokrotnym wyborem wartości
        widgets.interact(
            self._barplot, 
            years=widgets.SelectMultiple(options=self._years, description='Rok:', value=(self.default_year,)), 
            months=widgets.SelectMultiple(options=self._months, description='Miesiąc:', value=(self.default_month,)),
            rodzaje=widgets.SelectMultiple(options=self._rodzaje, description='Rodzaj:', value=(self.default_rodzaj,)),
            kategorie=widgets.SelectMultiple(options=self._kategorie, description='Kategoria:', value=self.default_kategoria, 
                                             layout=widgets.Layout(height='200px'))  # Ustalanie wysokości widgetu
        )


In [9]:
class WidgetBarPlot_lastNMonths():
    def __init__(self, data_in):
        self.data_in = data_in
        self._data_params()
    
    def _data_params(self):
        self._categories = sorted(list(set(self.data_in['Kategoria'].unique())))
        self._types = sorted(list(set(self.data_in['Rodzaj'].unique())))
        self._rodzaj = ("Debit",)

        # Znalezienie najdłuższej kategorii
        self._max_length = max(self._categories, key=len)

    def _barplot_lastn(self, categories, last_n_months, rodzaj):
        # Pobieranie najnowszej dostępnej daty
        latest_year = max(self.data_in['Rok'])
        latest_month = max(self.data_in[self.data_in['Rok'] == latest_year]["Miesiac"])
        
        # Określenie zakresu daty na podstawie last_n_months
        end_month = latest_month
        end_year = latest_year
        
        start_month = latest_month - last_n_months + 1
        start_year = latest_year
        while start_month <= 0:
            start_month += 12
            start_year -= 1

        # Filtrujemy dane według wybranego zakresu czasu, rodzaju i kategorii
        filtered_data = self.data_in[
            ((self.data_in['Rok'] > start_year) | 
            ((self.data_in['Rok'] == start_year) & (self.data_in['Miesiac'] >= start_month)) &
            ((self.data_in['Rok'] < end_year) |
            ((self.data_in['Rok'] == end_year) & (self.data_in['Miesiac'] <= end_month)))) &
            (self.data_in['Rodzaj'].isin(rodzaj)) &
            (self.data_in['Kategoria'].isin(categories))
        ]

        # Sumowanie danych dla wybranych kategorii
        summed_data = filtered_data.groupby(['Rok', 'Miesiac']).agg({'Suma': 'sum'}).reset_index()
        
        # Dodawanie kolumny "rok-miesiąc" do summed_data
        summed_data['Rok-Miesiac'] = summed_data['Rok'].astype(str) + "-" + summed_data['Miesiac'].astype(str).str.zfill(2)
        
        plt.figure(figsize=(15, 8))
        
        # Tworzenie wykresu słupkowego
        ax = sns.barplot(data=summed_data, x='Rok-Miesiac', y='Suma')
        
        # Dodawanie etykiet do słupków
        for p in ax.patches:
            ax.annotate(f'{p.get_height():.0f}', 
                        (p.get_x() + p.get_width() / 2., p.get_height() - p.get_height()/10), 
                        ha='center', va='center', 
                        xytext=(0, -10), 
                        textcoords='offset points')
        
        plt.title('Miesięczne wpływy/koszty w podziale na kategorie')
        plt.tight_layout()
        
        plt.show()


    def widget_plot(self):
    # Uruchamiam interaktywną wizualizację
        widgets.interact(
            self._barplot_lastn, 
            categories=widgets.SelectMultiple(
                options=self._categories, 
                description='Kategoria:', 
                layout=widgets.Layout(width=f'{len(self._max_length) * 8 + 20}px', height=f'{int(len(self._categories)*25*0.85)}px'),
                value=self._categories  # Domyślnie zaznacz wszystkie kategorie
            ),
            last_n_months=widgets.Dropdown(options=[3, 6, 9, 12, 18, 24], description='Ostatnie miesiące:', value=6),
            rodzaj=widgets.SelectMultiple(options=self._types, description='Rodzaj:', value=self._rodzaj)
        )


In [10]:
class PivotTimeRange():
    def __init__(self, data_in):
        self.data_in = data_in
        self._data_filters()
        self._widgets()
    
    def _data_filters(self):
        self.data_in['Rok-Miesiac'] = self.data_in['Rok'].astype(str) + '-' + self.data_in['Miesiac'].astype(str).str.zfill(2)
        
        # Ustalanie indeksów dla najnowszego roku w danych
        self.current_year = self.data_in['Rok'].max()
        self.start_idx = self.data_in[self.data_in['Rok'] == self.current_year]['Rok-Miesiac'].min()
        self.end_idx = self.data_in[self.data_in['Rok'] == self.current_year]['Rok-Miesiac'].max()


    def _widgets(self):
        # Widgety
        self.date_range = widgets.SelectionRangeSlider(
            options=sorted(self.data_in['Rok-Miesiac'].unique()),
            index=(self.data_in['Rok-Miesiac'].unique().tolist().index(self.start_idx), 
                self.data_in['Rok-Miesiac'].unique().tolist().index(self.end_idx)),
            description='Zakres dat:',
            continuous_update=False,
            layout=widgets.Layout(width='500px')
        )

        self.category_selector = widgets.SelectMultiple(
            options=sorted(self.data_in['Kategoria'].unique()),
            value=list(self.data_in['Kategoria'].unique()),
            description='Kategoria:',
            continuous_update=False,
            rows=len(self.data_in['Kategoria'].unique()),
            layout=widgets.Layout(width='1000px')
        )

        self.type_selector = widgets.SelectMultiple(
            options=sorted(self.data_in['Rodzaj'].unique()),
            value=['Debit'],
            description='Rodzaj:',
            continuous_update=False,
            rows=len(self.data_in['Rodzaj'].unique())
        )
        self.output = widgets.Output()

    def _update_table(self, change=None):
        with self.output:
            clear_output(wait=True)
            filtered_data = self.data_in[
                (self.data_in['Rok-Miesiac'] >= self.date_range.value[0]) &
                (self.data_in['Rok-Miesiac'] <= self.date_range.value[1]) &
                (self.data_in['Kategoria'].isin(self.category_selector.value)) &
                (self.data_in['Rodzaj'].isin(self.type_selector.value))
            ]
            pivot = filtered_data.pivot_table(index='Kategoria', columns='Rok-Miesiac', values='Suma', aggfunc='sum').fillna(0).sort_index()
            
            # Zaokrąglenie do 2 miejsc po przecinku
            pivot = pivot.round(2)
            
            # Dodanie kolumny "Sum_total"
            pivot['Sum_total'] = pivot.sum(axis=1)
            
            # Dodanie wiersza "TOTAL"
            pivot.loc['TOTAL'] = pivot.sum()
            
            display(pivot)

    def pivot_calc(self):
        self.date_range.observe(self._update_table, names='value')
        self.category_selector.observe(self._update_table, names='value')
        self.type_selector.observe(self._update_table, names='value')

        display_widgets = widgets.VBox([self.date_range, self.category_selector, self.type_selector, self.output])
        display(display_widgets)
        self._update_table()


In [11]:
filename = 'Wydatki_2021.xlsx'
# file = os.path.join(DataFolderPath, 'Private', filename)


In [12]:
# Unused SQL Queries - move to beggining


# 3. Przygotowanie agregacji danych
# Miesięczne wpływy
# query_monthly_income = "SELECT Rok, Miesiac, Kategoria, SUM(Cena) AS Wplywy FROM transactions WHERE Rodzaj = 'Credit' GROUP BY Rok, Miesiac, Kategoria"
# monthly_income_by_type = pd.read_sql(query_monthly_income, conn)

# Miesięczne wydatki
# query_monthly_expenses = "SELECT Rok, Miesiac, Kategoria, SUM(Cena) AS Wydatki FROM transactions WHERE Rodzaj = 'Debit' GROUP BY Rok, Miesiac, Kategoria"
# monthly_expenses_by_type = pd.read_sql(query_monthly_expenses, conn)

# Miesięczne przepływy
# query_monthly_flow = "SELECT Rok, Miesiac, SUM(Case When Rodzaj = 'Credit' THEN Cena ELSE 0 END) as Wplywy, SUM(Case When Rodzaj = 'Debit' THEN Cena ELSE 0 END) as Wydatki FROM transactions GROUP BY Rok, Miesiac"
# monthly_flow = pd.read_sql(query_monthly_flow, conn)
# monthly_flow["Przeplywy"] = monthly_flow["Wplywy"] + monthly_flow["Wydatki"]

# Analiza miesięcznej struktury wydatków
# monthly_expenses_by_type['Procent'] = monthly_expenses_by_type.groupby(['Rok', 'Miesiac'])['Wydatki'].transform(lambda x: x / x.sum() * 100)

# 4. Przygotowanie danych kumulatywnych
# Kumulacyjne wpływy
# monthly_income_by_type['Kumulacyjne_Wplywy'] = monthly_income_by_type.groupby(['Rok', 'Kategoria'])['Wplywy'].cumsum()

# Kumulacyjne wydatki
# monthly_expenses_by_type['Kumulacyjne_Wydatki'] = monthly_expenses_by_type.groupby(['Rok', 'Kategoria'])['Wydatki'].cumsum()

# Kumulacyjne przepływy
# monthly_flow['Kumulacyjne_Wplywy'] = monthly_flow.groupby('Rok')['Wplywy'].cumsum()
# monthly_flow['Kumulacyjne_Wydatki'] = monthly_flow.groupby('Rok')['Wydatki'].cumsum()
# monthly_flow['Kumulacyjne_Przeplywy'] = monthly_flow['Kumulacyjne_Wplywy'] + monthly_flow['Kumulacyjne_Wydatki']

# Kumulacyjna analiza struktury wydatków
# cumulative_expenses_sum = monthly_expenses_by_type.groupby(['Rok', 'Miesiac'])['Kumulacyjne_Wydatki'].sum().reset_index()
# cumulative_expenses_sum = cumulative_expenses_sum.rename(columns={'Kumulacyjne_Wydatki': 'Suma_Kumulacyjnych_Wydatkow'})
# cumulative_expenses_structure = pd.merge(monthly_expenses_by_type, cumulative_expenses_sum, on=['Rok', 'Miesiac'])
# cumulative_expenses_structure['Procent_Kumulacyjny'] = cumulative_expenses_structure['Kumulacyjne_Wydatki'] / cumulative_expenses_structure['Suma_Kumulacyjnych_Wydatkow'] 


In [27]:
wydatki = import_bills(filename = 'Wydatki_2021.xlsx', sheet_name = 'Wydatki')


c:\Users\tgusc\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


ValueError: could not convert string to float: ' '

In [26]:
wydatki_wspolne = import_bills(filename = 'Wydatki_2021.xlsx', sheet_name = 'Wydatki_KontoWspolne')


        LP       Data                               Przedmiot     Cena Waluta  \
1655  1599 2024-03-14                           Uber Eats KFC   -84.55    PLN   
1656  1600 2024-03-15  Dieta kukuła LowCarb 2500kcal - 24 dni -2331.06    PLN   
1657  1601 2024-03-16                        Mens cut napiwek   -20.00    PLN   
1658  1602 2024-03-16          player - wypożyczenie Whiplash   -10.00    PLN   
1659  1603 2024-03-08    Media Expert - oczyszczacz powietrza -1799.00    PLN   

                Sklep    Kto    Potrzeba                    Kategoria  \
1655        Uber Eats  Tomek    Potrzeba  jedzenie i picie: zamawiane   
1656            Tomek  Tomek    Potrzeba             jedzenie i picie   
1657  Mens cut barber  Tomek    Potrzeba  higiena, kosmetyki, fryzjer   
1658        player.pl  Tomek  Zachcianka      rozrywka i przyjemności   
1659     Media Expert  Tomek    Potrzeba      inne niezbędne potrzeby   

         Grupa   Rok  Miesiac Rodzaj Komentarz  
1655  Grupa IV  2024     

c:\Users\tgusc\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
monthly_wydatki = sql_sum(wydatki)


       Rok  Miesiac Rodzaj                       Kategoria     Suma
1056  2023       10  Debit                      suplementy -1371.25
1057  2023       10  Debit                       transport  -270.40
1058  2023       10  Debit                         ubranie   -73.99
1059  2023       10  Debit          wspólny rachunek - ada -9224.13
1060  2023       10  Debit  własny rozwój (kursy, zajęcia) -1817.99


In [15]:
monthly_wspolne = sql_sum(wydatki_wspolne)


      Rok  Miesiac Rodzaj                       Kategoria    Suma
397  2024        3  Debit                         prezent -519.93
398  2024        3  Debit         rozrywka i przyjemności -515.89
399  2024        3  Debit                       szastusia  -73.70
400  2024        3  Debit                            taxi -198.91
401  2024        3  Debit  własny rozwój (kursy, zajęcia)  -60.00


In [25]:
monthly_wspolne[(monthly_wspolne['Miesiac'] < 4) & (monthly_wspolne['Rok'] == 2024)]

,Rok,Miesiac,Rodzaj,Kategoria,Suma
349,2024,1,Debit,czynsz za mieszkanie,-1890.52
350,2024,1,Debit,hazard,-100.00
351,2024,1,Credit,inne dochody,15649.62
352,2024,1,Debit,"inne raty, składki i podatki",-135.52
353,2024,1,Debit,jedzenie i picie,-2617.71
354,2024,1,Debit,jedzenie i picie: na mieście,-113.50
355,2024,1,Debit,jedzenie i picie: niezdrowe,-27.60
356,2024,1,Debit,jedzenie i picie: zamawiane,-554.47
357,2024,1,Debit,koszty leczenia i leki,-159.80
358,2024,1,Debit,lokowanie nadwyżek,-1.00


In [ ]:
# https://www.google.com/search?client=firefox-b-d&q=vs+code+interactive+plots+in+python#ip=1


In [ ]:
wydatki_barplot = WidgetBarPlot(data_in = monthly_wydatki)
wydatki_barplot.widget_plot()


interactive(children=(SelectMultiple(description='Rok:', index=(4,), options=(2019, 2020, 2021, 2022, 2023), v…

In [ ]:
wspolne_barplot = WidgetBarPlot(data_in = monthly_wspolne)
wspolne_barplot.widget_plot()


In [ ]:
wydatki_lastN = WidgetBarPlot_lastNMonths(data_in = monthly_wydatki)
wydatki_lastN.widget_plot()


interactive(children=(SelectMultiple(description='Kategoria:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12…

In [ ]:
wspolne_lastN = WidgetBarPlot_lastNMonths(data_in = monthly_wspolne)
wspolne_lastN.widget_plot()


interactive(children=(SelectMultiple(description='Kategoria:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12…

In [ ]:
wydatki_pivot = PivotTimeRange(data_in = monthly_wydatki)
wydatki_pivot.pivot_calc()


In [ ]:
wspolne_pivot = PivotTimeRange(data_in = monthly_wspolne)
wspolne_pivot.pivot_calc()


In [ ]:
# DAĆ NET FLOW MIESIĘCZNY? JAK ZNIWELOWAĆ PRZEPŁYWY ŻEBY ZNAĆ TYLKO ZYSK/STRATĘ PER MIESIĄC?
# JAKIE DECYZJE PODEJMOWAĆ? 
# GDZIE NAJWIĘKSZE WYDATKI?
# GDZIE MOGĘ NAJWIĘCEJ OSZCZĘDZIĆ?


In [ ]:
# Wykres miesięcznych przepływów
plt.figure(figsize=(14, 7))
sns.lineplot(data=monthly_flow, x='Miesiac', y='Przeplywy', marker="o")
plt.title('Miesięczne przepływy')
plt.tight_layout()
plt.show()


NameError: name 'monthly_flow' is not defined

<Figure size 1400x700 with 0 Axes>

In [ ]:
# Stworzenie lokalnej bazy danych SQLite
conn = sqlite3.connect('transactions_db_temp.sqlite')
df.to_sql('transactions', conn, if_exists='replace', index=False)

# Przygotowanie agregacji danych
query_monthly_expenses = """
SELECT Rok, Miesiac, Kategoria, SUM(Cena) AS Wydatki 
FROM transactions WHERE Rodzaj = 'Debit' 
GROUP BY Rok, Miesiac, Kategoria
"""

monthly_expenses_by_type = pd.read_sql(query_monthly_expenses, conn)

# Poprawione obliczenie procentowego udziału kategorii w miesięcznych wydatkach
monthly_expenses_by_type['Procent'] = monthly_expenses_by_type.groupby(['Rok', 'Miesiac'])['Wydatki'].transform(lambda x: x / x.sum() * 100)

# Tworzenie monthly_expenses_structure
cumulative_expenses_sum = monthly_expenses_by_type.groupby(['Rok', 'Miesiac'])['Wydatki'].sum().reset_index()
cumulative_expenses_sum = cumulative_expenses_sum.rename(columns={'Wydatki': 'Suma_Wydatkow'})
monthly_expenses_structure = pd.merge(monthly_expenses_by_type, cumulative_expenses_sum, on=['Rok', 'Miesiac'])
monthly_expenses_structure['Procent_Struktury'] = monthly_expenses_structure['Wydatki'] / monthly_expenses_structure['Suma_Wydatkow'] * 100

monthly_expenses_structure.head(10)


,Rok,Miesiac,Kategoria,Wydatki,Procent,Suma_Wydatkow,Procent_Struktury
0,2019,11,alkohol,-120.86,9.804256,-1232.73,9.804256
1,2019,11,czynsz za mieszkanie,-517.74,41.999465,-1232.73,41.999465
2,2019,11,dowolne zakupy,-0.45,0.036504,-1232.73,0.036504
3,2019,11,"higiena, kosmetyki, fryzjer",-6.99,0.567034,-1232.73,0.567034
4,2019,11,"hobby, kino, teatr",-29.70,2.409287,-1232.73,2.409287
5,2019,11,jedzenie i picie,-185.25,15.027622,-1232.73,15.027622
6,2019,11,jedzenie i picie: na mieście,-182.33,14.790749,-1232.73,14.790749
7,2019,11,jedzenie i picie: niezdrowe,-16.57,1.344171,-1232.73,1.344171
8,2019,11,jedzenie i picie: zamawiane,-139.86,11.345550,-1232.73,11.345550
9,2019,11,pomoc innym,-24.37,1.976913,-1232.73,1.976913
